# Feature Generation 
## 1.for sampled_play
### 1.1 Frequency of time_period, average play_time of the same time_period 

In [1]:
import pandas as pd
import numpy as np
import datetime as dt

Get the sample_uid_label(8823) by merge sample_uid(8823) with sample_uid_label(8863)

In [161]:
s8823 = pd.read_csv("D:/MusicFile/sample_uid_316/sample_uid_8823.csv")
sl8862 = pd.read_csv("D:/MusicFile/sample_uid_label/sample_uid_316.csv")
sl8862.sample(8)
print(s8823.count(), sl8862.count())
sample_uid_label = s8823.merge(sl8863, on = 'uid', how = 'inner')

sample_uid_label.sample(8)

uid    8823
dtype: int64 uid      8862
churn    8862
dtype: int64


,uid,churn
215,168105024.0,1
1205,168320032.0,0
4318,154454656.0,1
4504,168472192.0,0
4060,168485248.0,0
3055,168773616.0,0
7968,167799664.0,0
1672,167769056.0,0


In [159]:
sample_uid_label.to_csv("D:/MusicFile/sample_uid_label8823.csv", header = True, index = False)

In [2]:
colname = ['uid', 'device', 'song_id', 'play_time', 'song_length', 'date', 'Churn']
datatype = {'uid': 'float64', 'device': 'object', 'song_id': 'float64', 'song_length': 'float64', 'date':'object', 'Churn': 'float64'}
sample_play = pd.read_csv("D:/MusicFile/sample_play_316.csv", header = 0, dtype = datatype, names = colname)

In [139]:
sample_play.sample(8)

,uid,device,song_id,play_time,song_length,date,Churn
1750966,168880640.0,ar,8071524.0,105.0,105.0,2017-04-04,0.0
2263033,154783952.0,ar,1043641.0,105.0,277.0,2017-03-09,1.0
3007123,168101632.0,ar,23485856.0,219.0,219.0,2017-03-31,1.0
3079592,168373360.0,ar,22251534.0,122.0,122.0,2017-04-11,0.0
4842318,167929168.0,ar,3188740.0,225.0,226.0,2017-03-31,0.0
1084272,168011376.0,ip,1116274.0,245.0,245.0,2017-04-28,0.0
2589122,167899632.0,ar,4221232.0,4.0,303.0,2017-04-23,0.0
761418,167579040.0,ar,5820733.0,3.0,251.0,2017-04-17,0.0


In [7]:
sample_play['date'] = pd.to_datetime(sample_play['date'])
sample_play.dtypes

uid                   float64
device                 object
song_id               float64
play_time             float64
song_length           float64
date           datetime64[ns]
Churn                 float64
dtype: object

In [8]:
sample_play.shape

(4955455, 7)

In [137]:
print('How many active user:',len(sample_play.uid[sample_play.Churn == 0].unique()))
print('How many churn user:',len(sample_play.uid[sample_play.Churn == 1].unique()))

How many active user: 4465
How many churn user: 4358


In [5]:

start = '2017-03-01'
end = '2017-04-29'
st,ed = pd.to_datetime(start), pd.to_datetime(end)
elapsed = ed-st

if elapsed > dt.timedelta(days = 1):
    print("Slept for > 1 day")

Slept for > 1 day


In [9]:
log_types = ['play', 'search', 'download']
window_list = [1,3,7,14,21]
snapshot_date = '2017-04-29'

In [142]:
def get_freq_avg(df, log_type, window, snapshot_date):
    snapshot_date = pd.to_datetime(snapshot_date)
    #df2 shows records inside window
    df2 = df.loc[(snapshot_date - df['date'])<= dt.timedelta(window), :]
    print('How many active/churn user after feature generation:',len(df2.uid[df2.Churn == 0].unique()), len(df2.uid[df2.Churn ==1].unique()))
    df_feature = df2.groupby('uid')['play_time'] \
                    .agg(['count', 'mean']) \
                    .rename(columns = {'count':'freq_' + log_type + '_last_' + str(window) + '_days',
                                       'mean' : 'avg_'+ log_type + 'time' + '_last_' + str(window) + '_days'})
    #df_feature = df_feature.rename(columns = {'date':})
    return df_feature

In [143]:
test1 = get_freq_avg(sample_play, 'play', window_list[0], snapshot_date)

How many active/churn user after feature generation: 2722 69


In [12]:
print(test1.columns, test1.shape[0])
test1.head()

Index(['freq_play_last_1_days', 'avg_playtime_last_1_days'], dtype='object') 2791


,freq_play_last_1_days,avg_playtime_last_1_days
uid,,
1901839.0,3,222.333333
2285308.0,11,107.818182
5597959.0,6,111.166667
7533076.0,4,198.000000
7910960.0,14,214.142857


In [144]:
def get_all_freq_avg(df, log_type, window_list, snapshot_time):
    type_feature = get_freq_avg(df, log_type, window_list[0], snapshot_date)
    for window in window_list[1:]:
        #what = 'freq and average time of' + log_type + '_last_' + str(window) + '_days'
        #print("Calculating", what)
        test_what = get_freq_avg(df, log_type, window, snapshot_date)
        #print(test_what.head())
        type_feature = pd.merge(test_what, type_feature, how='left', left_index=True, right_index=True)
    return type_feature

In [145]:
play_feature = get_all_freq_avg(sample_play, 'play', window_list, snapshot_date)
#since many churn user never show up,  ┓( ´∀` )┏

How many active/churn user after feature generation: 2722 69
How many active/churn user after feature generation: 3621 186
How many active/churn user after feature generation: 4085 389
How many active/churn user after feature generation: 4319 667
How many active/churn user after feature generation: 4397 891


In [15]:
print(play_feature.shape)
play_feature.sample(8)

(5288, 10)


,freq_play_last_21_days,avg_playtime_last_21_days,freq_play_last_14_days,avg_playtime_last_14_days,freq_play_last_7_days,avg_playtime_last_7_days,freq_play_last_3_days,avg_playtime_last_3_days,freq_play_last_1_days,avg_playtime_last_1_days
uid,,,,,,,,,,
167754432.0,828,168.347826,552.0,156.530797,274.0,147.536496,96.0,141.031250,15.0,204.600000
168097696.0,487,230.030801,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
168437440.0,1241,1947.092667,1083.0,1770.493998,85.0,146.282353,18.0,138.055556,12.0,91.666667
168266560.0,394,215.532995,120.0,165.016667,30.0,184.466667,NaN,NaN,NaN,NaN
168019680.0,602,156.431894,403.0,171.292804,253.0,157.094862,120.0,181.508333,26.0,220.038462
57545228.0,30,171.866667,5.0,207.800000,1.0,61.000000,1.0,61.000000,1.0,61.000000
167624816.0,16,8.187500,16.0,8.187500,NaN,NaN,NaN,NaN,NaN,NaN
167935584.0,206,198.014563,123.0,222.747967,19.0,511.315789,NaN,NaN,NaN,NaN


In [166]:
# add dropped churn user back
#sample_uid_label = sample_uid_label.set_index('uid')
#sample_uid_label.sample(8)
play_feature_8823 = play_feature.join(sample_uid_label, how = 'outer')

In [168]:
play_feature_8823.sample(8)

(8823, 11)


,freq_play_last_21_days,avg_playtime_last_21_days,freq_play_last_14_days,avg_playtime_last_14_days,freq_play_last_7_days,avg_playtime_last_7_days,freq_play_last_3_days,avg_playtime_last_3_days,freq_play_last_1_days,avg_playtime_last_1_days,churn
uid,,,,,,,,,,,
154605792.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
167927712.0,1048.0,233.916031,508.0,302.438976,350.0,327.897143,106.0,486.867925,18.0,51.277778,0
167935968.0,170.0,20.688235,7.0,155.857143,5.0,151.800000,4.0,149.250000,NaN,NaN,0
168456864.0,327.0,115.339450,211.0,85.635071,85.0,138.411765,34.0,100.294118,NaN,NaN,0
154677904.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
154590928.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
154622032.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
154418768.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [169]:
print(len(sample_play.uid.unique()), play_feature.shape[0], play_feature_8823.shape[0])

8823 5288 8823


### 1.2. device feature

In [18]:
sample_play[['uid', 'device', 'Churn']].drop_duplicates().shape

(12618, 3)

In [19]:
sample_play[['uid', 'Churn']].drop_duplicates().shape

(8823, 2)

Adding device as another groupby condition causing significant increase, we can assume that some users have MusicBox installed in more than one device, such as their android phone and ipad.

Create a new value for user who have both device

In [21]:
device_count = sample_play[['uid', 'device']].drop_duplicates() \
                                              .groupby('uid')['device'] \
                                              .agg(['max', 'count'])

In [100]:
device_count.head()

,max,count
uid,,
400904.0,ar,1
838955.0,ar,1
971787.0,ar,1
1264383.0,ar,1
1901839.0,ar,1


In [22]:
device_count['device'] = np.where(device_count['count'] == 1, device_count['max'], 'both')

In [23]:
device_count['device'].value_counts()

ar      4862
both    3795
ip       166
Name: device, dtype: int64

As we assumed, there are 40% user have both device

In [24]:
device_feature = sample_play[['uid', 'Churn']].drop_duplicates() \
                                               .reset_index(drop=True) \
                                               .set_index('uid') \
                                               .join(device_count['device'])

In [26]:
print(device_feature.shape)
device_feature.sample(8)

(8823, 2)


,Churn,device
uid,,
168540016.0,0.0,ar
154617296.0,1.0,both
154613056.0,1.0,both
168764656.0,0.0,both
154795248.0,1.0,both
96635472.0,0.0,ar
167999856.0,0.0,both
154549488.0,1.0,both


### 1.3. merge device_feature and play_feature

In [174]:
play_feature_all = play_feature_8823.join(device_feature,how = "left").drop('churn', 1)
print(play_feature_all.shape)
play_feature_all.sample(8)


(8823, 12)


,freq_play_last_21_days,avg_playtime_last_21_days,freq_play_last_14_days,avg_playtime_last_14_days,freq_play_last_7_days,avg_playtime_last_7_days,freq_play_last_3_days,avg_playtime_last_3_days,freq_play_last_1_days,avg_playtime_last_1_days,Churn,device
uid,,,,,,,,,,,,
168687520.0,173.0,130.560694,167.0,131.095808,NaN,NaN,NaN,NaN,NaN,NaN,0.0,both
168363200.0,818.0,156.286064,584.0,140.607877,287.0,205.822300,177.0,204.067797,87.0,162.850575,0.0,ar
136122752.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,ar
168340672.0,593.0,98.424958,300.0,101.143333,150.0,125.373333,81.0,143.086420,35.0,106.514286,0.0,ar
168543104.0,273.0,210.846646,196.0,213.755102,99.0,220.202020,35.0,225.000000,17.0,225.294118,0.0,both
154116704.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,ar
168997408.0,18.0,165.611111,10.0,289.300000,NaN,NaN,NaN,NaN,NaN,NaN,1.0,ar
128491368.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,ar


In [138]:
#################Original#################
print('How many active user after feature generation:',len(play_feature_all[play_feature_all.Churn == 0]))
print('How many churn user after feature generation:',len(play_feature_all[play_feature_all.Churn == 1]))

How many active user after feature generation: 4397
How many churn user after feature generation: 891


In [175]:
###################Adjusted##################
print('How many active user after feature generation:',len(play_feature_all[play_feature_all.Churn == 0]))
print('How many churn user after feature generation:',len(play_feature_all[play_feature_all.Churn == 1]))

How many active user after feature generation: 4465
How many churn user after feature generation: 4358


### 1.4 complete_degree
check average music play complete degree of each user: avg(play_time/song_length)


In [29]:
sample_play.sample(8)

,uid,device,song_id,play_time,song_length,date,Churn
4428001,168738928.0,ip,658147.0,0.0,298.0,2017-04-10,1.0
4458463,154485536.0,ip,20794644.0,204.0,204.0,2017-03-04,1.0
385254,168584224.0,ar,17081440.0,0.0,333.0,2017-03-31,0.0
4089684,167957136.0,ar,6748381.0,1209.0,1209.0,2017-04-11,0.0
3777979,167845184.0,ar,1381671.0,12.0,166.0,2017-04-24,0.0
1931891,168217120.0,ip,4858877.0,86.0,195.0,2017-04-25,0.0
1890034,165995456.0,ar,22871364.0,21.0,86.0,2017-04-01,1.0
469288,167887840.0,ar,7103520.0,4.0,280.0,2017-04-01,0.0


In [60]:
music_complete = sample_play.groupby('uid')[['play_time', 'song_length']].mean()
print('How many user play time =0:', music_complete[music_complete.play_time ==0].count())
zero_uid = music_complete[music_complete.play_time ==0].index.tolist()
#if these users' average play time is 0....drop them? when clean play.log, I set play_time >=0, because I think for one song, the user
#just skip it, but not for all songs. If so, those user should be dropped.
print(zero_uid)
print(music_complete.head())

How many user play time =0: play_time      25
song_length    25
dtype: int64
[400904.0, 4334792.0, 8955891.0, 18205710.0, 61622544.0, 67449880.0, 106513760.0, 116009376.0, 124488288.0, 127544832.0, 128475592.0, 133089264.0, 135697680.0, 153251792.0, 154314656.0, 154882240.0, 155123760.0, 155320656.0, 155400256.0, 158236640.0, 161995536.0, 166548304.0, 167466016.0, 167848496.0, 169048464.0]
            play_time  song_length
uid                               
400904.0     0.000000   238.240000
838955.0   167.769231   222.307692
971787.0    32.500000   243.000000
1264383.0   87.666667   234.666667
1901839.0  160.455806   198.656846


In [121]:
complete_precent = music_complete.play_time/music_complete.song_length
complete_precent = pd.DataFrame(complete_precent, columns = ['precent'])
complete_precent.reset_index(inplace = True)
complete_precent.head()
#there are 25 user with play_time =0, 928 user with play/song_length >1.

,uid,precent
0,400904.0,0.000000
1,838955.0,0.754671
2,971787.0,0.133745
3,1264383.0,0.373580
4,1901839.0,0.807703


### 1.4. save play_feature_all

In [176]:
#this is 8823, remember to fillna with 0
play_feature_all.to_csv("D:/MusicFile/play_feature_8823.csv")

## 2. for search and download.log
### 2.1 frequency feature generation for search

In [78]:
sampled_search = pd.read_csv("D:/MusicFile/sample_search_316.csv")
sampled_search.sample(8)

,uid,device,file_name,date,record_time
251201,168930432.0,ar,20170423_3_search.log,2017-04-23T00:00:00.000-04:00,2017-04-23T18:13:34.000-04:00
9317,167656192.0,ip,20170331_1_search.log,2017-03-31T00:00:00.000-04:00,2017-03-31T19:50:34.000-04:00
289063,168733904.0,ar,20170330_3_search.log,2017-03-30T00:00:00.000-04:00,2017-03-30T23:17:28.000-04:00
160479,168958720.0,ip,20170413_3_search.log,2017-04-13T00:00:00.000-04:00,2017-04-14T00:10:47.000-04:00
260034,168328640.0,ar,20170422_2_search.log,2017-04-22T00:00:00.000-04:00,2017-04-22T07:48:51.000-04:00
459521,168980560.0,ar,20170331_3_search.log,2017-03-31T00:00:00.000-04:00,2017-03-31T20:59:27.000-04:00
404774,167814544.0,ar,20170415_1_search.log,2017-04-15T00:00:00.000-04:00,2017-04-15T22:22:59.000-04:00
244018,168777920.0,ar,20170408_3_search.log,2017-04-08T00:00:00.000-04:00,2017-04-08T21:53:17.000-04:00


In [79]:
sampled_search['date'] = pd.to_datetime(sampled_search['date'].str.slice(0,10))

In [80]:
sampled_search.sample(8)

,uid,device,file_name,date,record_time
174544,168032048.0,ar,20170405_1_search.log,2017-04-05,2017-04-05T10:43:39.000-04:00
331657,154564032.0,ar,1_1_search.log,2017-03-01,2017-03-01T16:10:14.000-05:00
93294,154794272.0,ar,1_1_search.log,2017-03-01,2017-03-01T23:29:15.000-05:00
468830,168879056.0,ar,20170330_3_search.log,2017-03-30,2017-03-30T20:14:16.000-04:00
337112,169040624.0,ar,20170419_3_search.log,2017-04-19,2017-04-19T09:31:00.000-04:00
367342,154662560.0,ar,1_1_search.log,2017-03-01,2017-03-01T19:13:47.000-05:00
99225,168376224.0,ar,20170420_2_search.log,2017-04-20,2017-04-20T18:24:33.000-04:00
213695,167907184.0,ar,20170416_1_search.log,2017-04-16,2017-04-16T23:44:42.000-04:00


In [6]:
sampled_search.dtypes

Unnamed: 0              int64
uid                   float64
device                 object
date           datetime64[ns]
record_time            object
dtype: object

In [84]:
sampled_search = sampled_search[['uid', 'device', 'date']]
len(sampled_search.uid.unique())

7714

In [85]:
sampled_search.to_csv("D:/MusicFile/sampled_search_316new.csv", header = True)

In [ ]:
'''
def get_freq_avg(df, log_type, window, snapshot_date):
    snapshot_date = pd.to_datetime(snapshot_date)
    df2 = df.loc[(snapshot_date - df['date'])<= dt.timedelta(window), :]
    df_feature = df2.groupby('uid')['play_time'] \
    .agg(['count', 'mean']) \
    .rename(columns = {'count':'freq_' + log_type + '_last_' + str(window) + '_days',
                       'mean' : 'avg_'+ log_type + 'time' + '_last_' + str(window) + '_days'})
    return df_feature

def get_all_freq(df, log_type, window_list, snapshot_time):
    type_feature = get_freq_avg(df, log_type, window_list[0], snapshot_date)
    for window in window_list[1:]:
        test_what = get_freq_avg(df, log_type, window, snapshot_date
        type_feature = pd.merge(test_what, type_feature, how='left', left_index=True, right_index=True)
    return type_feature


log_types = ['play', 'search', 'download']
window_list = [1,3,7,14,21]
snapshot_date = '2017-04-29'

'''


In [86]:
log_types = ['play', 'search', 'download']
window_list = [1,3,7,14,21]
snapshot_date = '2017-04-29'

In [87]:
def get_freq(df, log_type, window, snapshot_time):
    snapshot_date = pd.to_datetime(snapshot_time)
    df2 = df.loc[(snapshot_date - df['date'])<= dt.timedelta(window), :]
    df_feature = df2.groupby('uid')['date'] \
                    .agg(['count']) \
                    .rename(columns = {'count':'freq_' + log_type + '_last_' + str(window) + '_days'})
    
    return df_feature

In [9]:
sest = get_freq(sampled_search, 'search', window_list[0], snapshot_date)
sest.head()

,freq_search_last_1_days
uid,
1901839.0,5
2163514.0,1
2285308.0,1
3752046.0,3
5597959.0,18


In [88]:
def get_all_freq(df, log_type, window_list, snapshot_time):
    type_feature = get_freq(df, log_type, window_list[0], snapshot_date)
    for window in window_list[1:]:
        test_what = get_freq(df, log_type, window, snapshot_date)
        type_feature = pd.merge(test_what, type_feature, how='left', left_index=True, right_index=True)
    return type_feature

In [89]:
search_feature = get_all_freq(sampled_search, 'search', window_list, snapshot_date)

In [91]:
search_feature.sample(8)
#notice: uid now is the row index, we need it as a column uid, but when save to csv, it automatically change to a column

,freq_search_last_21_days,freq_search_last_14_days,freq_search_last_7_days,freq_search_last_3_days,freq_search_last_1_days
uid,,,,,
154383536.0,1,1.0,1.0,NaN,NaN
168291328.0,183,155.0,118.0,94.0,94.0
169127536.0,20,16.0,6.0,6.0,6.0
164304880.0,18,16.0,16.0,16.0,16.0
168250144.0,65,53.0,40.0,36.0,35.0
168078656.0,12,7.0,6.0,6.0,6.0
168646448.0,32,24.0,11.0,11.0,11.0
168926224.0,110,110.0,86.0,76.0,73.0


In [92]:
#same as  play_feature, there are some churn user never show up during the window time
search_feature.shape

(4633, 5)

In [93]:
search_feature.to_csv("D:/MusicFile/search_feature_316.csv", header = True)

### 2.2 frequency feature generation for download

In [95]:
sampled_download = pd.read_csv("D:/MusicFile/sample_download_316.csv")
sampled_download['date'] = pd.to_datetime(sampled_download['date'].str.slice(0,10))
sampled_download.sample(8)

,uid,device,song_id,date
250139,168495920.0,ar,94675.0,2017-04-26
206818,168714048.0,ar,6368802.0,2017-04-16
73410,168471200.0,ar,3197116.0,2017-04-03
218557,168948688.0,ar,4269836.0,2017-05-11
204064,168794400.0,ar,519479.0,2017-03-31
199924,167952240.0,ar,5070737.0,2017-05-09
201782,49423096.0,ar,4799485.0,2017-04-07
279274,168518592.0,ar,3328045.0,2017-03-30


In [16]:
dest = get_freq(sampled_download, 'download', window_list[0], snapshot_date)
dest.head()

,freq_download_last_1_days
uid,
2280694.0,5
3752046.0,1
7910960.0,3
8853855.0,2
14780539.0,27


In [96]:
download_feature = get_all_freq(sampled_download, 'download', window_list, snapshot_date)

In [98]:
download_feature.sample(8)

,freq_download_last_21_days,freq_download_last_14_days,freq_download_last_7_days,freq_download_last_3_days,freq_download_last_1_days
uid,,,,,
167662048.0,9,8.0,7.0,7.0,2.0
168883456.0,80,59.0,36.0,22.0,4.0
167726464.0,8,7.0,6.0,6.0,2.0
167643488.0,164,164.0,164.0,164.0,164.0
167958544.0,1,1.0,1.0,NaN,NaN
168196624.0,38,38.0,8.0,8.0,8.0
168695184.0,8,8.0,6.0,6.0,6.0
168239504.0,6,5.0,3.0,3.0,3.0


In [99]:
download_feature.shape

(3899, 5)

In [100]:
download_feature.to_csv("D:/MusicFile/download_feature_316.csv", header = True)

In [38]:
#join search and download
ds = pd.merge(search_feature, download_feature, left_on = 'uid', right_on = 'uid',  how = 'left')
ds.sample(10)

,uid,freq_search_last_21_days,freq_search_last_14_days,freq_search_last_7_days,freq_search_last_3_days,freq_search_last_1_days,freq_download_last_21_days,freq_download_last_14_days,freq_download_last_7_days,freq_download_last_3_days,freq_download_last_1_days
3628,168710656.0,3,NaN,NaN,NaN,NaN,6.0,2.0,NaN,NaN,NaN
3654,168719200.0,17,14.0,10.0,8.0,8.0,11.0,8.0,4.0,3.0,3.0
130,125439600.0,2,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4513,169007776.0,58,48.0,40.0,39.0,35.0,162.0,153.0,152.0,10.0,9.0
3093,168515184.0,35,10.0,8.0,6.0,6.0,4.0,NaN,NaN,NaN,NaN
2288,168248544.0,8,5.0,5.0,5.0,5.0,31.0,2.0,2.0,2.0,2.0
3017,168489712.0,18,13.0,11.0,8.0,8.0,4.0,3.0,3.0,3.0,3.0
2735,168397136.0,15,1.0,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN
3874,168789456.0,14,12.0,7.0,1.0,1.0,7.0,5.0,5.0,1.0,NaN
1960,168130448.0,90,65.0,37.0,28.0,28.0,43.0,9.0,8.0,8.0,8.0


### 3.Join play, search, download feature
Since our analysis based on play, so if user doesn't have search, download, just plugged in NaN
play_feature_all has 5395 user, search has 4645, download has 3906

In [22]:
print(4645/5395, 3906/5395)

0.860982391102873 0.7240037071362373


In [177]:
#reload type_feature

play_feature_all = pd.read_csv("D:/MusicFile/play_feature_8823.csv")
search_feature = pd.read_csv("D:/MusicFile/search_feature_316.csv")
download_feature = pd.read_csv("D:/MusicFile/download_feature_316.csv")

In [190]:
print("play_feature:",play_feature_all.shape)
print("search_feature:",search_feature.shape)
print("download_feature:",download_feature.shape)

play_feature: (8823, 13)
search_feature: (4633, 6)
download_feature: (3899, 6)


In [191]:
all_feature = play_feature_all.merge(search_feature, on = 'uid', how = "left") \
                              .merge(download_feature, on = 'uid', how = 'left')
    

In [192]:
all_feature.sample(10)    

,uid,freq_play_last_21_days,avg_playtime_last_21_days,freq_play_last_14_days,avg_playtime_last_14_days,freq_play_last_7_days,avg_playtime_last_7_days,freq_play_last_3_days,avg_playtime_last_3_days,freq_play_last_1_days,...,freq_search_last_21_days,freq_search_last_14_days,freq_search_last_7_days,freq_search_last_3_days,freq_search_last_1_days,freq_download_last_21_days,freq_download_last_14_days,freq_download_last_7_days,freq_download_last_3_days,freq_download_last_1_days
6309,168298288.0,270.0,132.607407,30.0,120.933333,19.0,109.684211,15.0,107.533333,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1884,154582128.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1816,154567200.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1265,154464592.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
152,44936528.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2789,154768144.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3011,154810816.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4814,167832720.0,101.0,568.059406,44.0,1076.750000,24.0,805.666667,23.0,831.000000,11.0,...,32.0,16.0,3.0,3.0,3.0,13.0,8.0,2.0,2.0,2.0
2509,154708864.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8532,168971088.0,90.0,97.488889,66.0,90.696970,26.0,104.769231,19.0,139.315789,5.0,...,116.0,113.0,96.0,96.0,93.0,NaN,NaN,NaN,NaN,NaN


In [181]:
all_feature.shape

(8823, 23)

In [182]:
all_feature.isnull().sum()

uid                              0
freq_play_last_21_days        3535
avg_playtime_last_21_days     3535
freq_play_last_14_days        3837
avg_playtime_last_14_days     3837
freq_play_last_7_days         4349
avg_playtime_last_7_days      4349
freq_play_last_3_days         5016
avg_playtime_last_3_days      5016
freq_play_last_1_days         6032
avg_playtime_last_1_days      6032
Churn                            0
device                           0
freq_search_last_21_days      4190
freq_search_last_14_days      4437
freq_search_last_7_days       4779
freq_search_last_3_days       5012
freq_search_last_1_days       5108
freq_download_last_21_days    4924
freq_download_last_14_days    5300
freq_download_last_7_days     5732
freq_download_last_3_days     6007
freq_download_last_1_days     6162
dtype: int64

In [193]:
all_feature.columns

Index(['uid', 'freq_play_last_21_days', 'avg_playtime_last_21_days',
       'freq_play_last_14_days', 'avg_playtime_last_14_days',
       'freq_play_last_7_days', 'avg_playtime_last_7_days',
       'freq_play_last_3_days', 'avg_playtime_last_3_days',
       'freq_play_last_1_days', 'avg_playtime_last_1_days', 'Churn', 'device',
       'freq_search_last_21_days', 'freq_search_last_14_days',
       'freq_search_last_7_days', 'freq_search_last_3_days',
       'freq_search_last_1_days', 'freq_download_last_21_days',
       'freq_download_last_14_days', 'freq_download_last_7_days',
       'freq_download_last_3_days', 'freq_download_last_1_days'],
      dtype='object')

In [194]:
#reorder columns
all_feature = all_feature[['uid','device', 'freq_play_last_21_days', 'avg_playtime_last_21_days',
       'freq_play_last_14_days', 'avg_playtime_last_14_days',
       'freq_play_last_7_days', 'avg_playtime_last_7_days',
       'freq_play_last_3_days', 'avg_playtime_last_3_days',
       'freq_play_last_1_days', 'avg_playtime_last_1_days', 
       'freq_search_last_21_days', 'freq_search_last_14_days',
       'freq_search_last_7_days', 'freq_search_last_3_days',
       'freq_search_last_1_days', 'freq_download_last_21_days',
       'freq_download_last_14_days', 'freq_download_last_7_days',
       'freq_download_last_3_days', 'freq_download_last_1_days', 'Churn']]

Since all NAN appears in search/download feature, NaN means there is no such record during this period, fillin with 0 is doable.

In [112]:
all_feature = all_feature.fillna(0)

## One_hot_encoding for device
sklearn can not accept numeric variables, for device, either use one-hot encoding or try model without device, save both and try later.

In [195]:
all_feature.sample(8)

,uid,device,freq_play_last_21_days,avg_playtime_last_21_days,freq_play_last_14_days,avg_playtime_last_14_days,freq_play_last_7_days,avg_playtime_last_7_days,freq_play_last_3_days,avg_playtime_last_3_days,...,freq_search_last_14_days,freq_search_last_7_days,freq_search_last_3_days,freq_search_last_1_days,freq_download_last_21_days,freq_download_last_14_days,freq_download_last_7_days,freq_download_last_3_days,freq_download_last_1_days,Churn
4177,167645552.0,both,243.0,174.259259,130.0,236.561538,61.0,240.819672,4.0,256.500000,...,3.0,3.0,2.0,2.0,30.0,28.0,28.0,28.0,28.0,0.0
5040,167905248.0,ar,49.0,120.551020,33.0,132.575758,1.0,210.000000,1.0,210.000000,...,1.0,NaN,NaN,NaN,7.0,1.0,NaN,NaN,NaN,0.0
4059,167617376.0,ar,99.0,188.000000,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,1.0,81.0,80.0,80.0,80.0,80.0,0.0
8601,168992992.0,ar,19.0,57.947368,19.0,57.947368,19.0,57.947368,NaN,NaN,...,8.0,8.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,0.0
2816,154775488.0,ar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2671,154745184.0,both,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4829,167839104.0,both,1799.0,170.148677,1231.0,185.941511,745.0,188.170470,227.0,161.762115,...,25.0,15.0,12.0,10.0,151.0,151.0,151.0,150.0,149.0,0.0
159,46472868.0,ar,4.0,122.000000,4.0,122.000000,4.0,122.000000,NaN,NaN,...,13.0,13.0,13.0,13.0,NaN,NaN,NaN,NaN,NaN,0.0


In [196]:
device_dummies = pd.get_dummies(all_feature.device)

In [197]:
device_dummies.sample(10)

,ar,both,ip
8573,1,0,0
2336,0,1,0
6142,1,0,0
4106,0,1,0
6986,0,1,0
6021,0,1,0
8230,0,1,0
6044,1,0,0
710,1,0,0
2442,0,1,0


In [198]:
#merge to the all_feature as all_feature_device_dummy
all_feature['device_ar'], all_feature['device_both'], all_feature['device_ip'] = device_dummies.ar, device_dummies.both, device_dummies.ip
all_feature_dd = all_feature.drop('device', 1)


In [199]:
all_feature_dd.sample(5)

,uid,freq_play_last_21_days,avg_playtime_last_21_days,freq_play_last_14_days,avg_playtime_last_14_days,freq_play_last_7_days,avg_playtime_last_7_days,freq_play_last_3_days,avg_playtime_last_3_days,freq_play_last_1_days,...,freq_search_last_1_days,freq_download_last_21_days,freq_download_last_14_days,freq_download_last_7_days,freq_download_last_3_days,freq_download_last_1_days,Churn,device_ar,device_both,device_ip
8556,168978192.0,298.0,232.684564,183.0,239.562842,102.0,244.107843,38.0,249.631579,14.0,...,23.0,7.0,7.0,7.0,7.0,7.0,0.0,1,0,0
1541,154519904.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0,1,0
1824,154568896.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0,1,0
602,146175264.0,6.0,180.500000,6.0,180.500000,6.0,180.500000,6.0,180.500000,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1,0,0
5290,167976800.0,685.0,32.363504,617.0,21.439222,308.0,21.035714,77.0,16.558442,NaN,...,1.0,36.0,NaN,NaN,NaN,NaN,0.0,1,0,0


In [200]:
#drop device as all_feature_no_d
all_feature_nd = all_feature_dd.drop(['device_ar', 'device_ip', 'device_both'],1)
all_feature_nd.sample(5)

,uid,freq_play_last_21_days,avg_playtime_last_21_days,freq_play_last_14_days,avg_playtime_last_14_days,freq_play_last_7_days,avg_playtime_last_7_days,freq_play_last_3_days,avg_playtime_last_3_days,freq_play_last_1_days,...,freq_search_last_14_days,freq_search_last_7_days,freq_search_last_3_days,freq_search_last_1_days,freq_download_last_21_days,freq_download_last_14_days,freq_download_last_7_days,freq_download_last_3_days,freq_download_last_1_days,Churn
3978,167598768.0,130.0,31911.976923,104.0,72.913462,NaN,NaN,NaN,NaN,NaN,...,12.0,6.0,6.0,6.0,1.0,1.0,NaN,NaN,NaN,0.0
2521,154710912.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
8024,168806032.0,301.0,138.777409,241.0,137.427386,220.0,138.495455,49.0,160.77551,4.0,...,82.0,72.0,55.0,48.0,2.0,NaN,NaN,NaN,NaN,0.0
3940,167588112.0,415.0,121.400000,94.0,194.638298,2.0,2.000000,2.0,2.00000,2.0,...,9.0,9.0,9.0,9.0,11.0,3.0,2.0,2.0,2.0,0.0
2213,154650144.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


# different version of features

In [201]:
#add precentage
all_feature_total = all_feature_dd.merge(complete_precent, on = 'uid', how = 'inner')
print(len(all_feature_total.uid.unique()))
all_feature_total.sample(8)

8823


,uid,freq_play_last_21_days,avg_playtime_last_21_days,freq_play_last_14_days,avg_playtime_last_14_days,freq_play_last_7_days,avg_playtime_last_7_days,freq_play_last_3_days,avg_playtime_last_3_days,freq_play_last_1_days,...,freq_download_last_21_days,freq_download_last_14_days,freq_download_last_7_days,freq_download_last_3_days,freq_download_last_1_days,Churn,device_ar,device_both,device_ip,precent
6270,168285616.0,520.0,70.894231,353.0,49.640227,293.0,38.887372,95.0,96.526316,50.0,...,3.0,3.0,2.0,1.0,1.0,0.0,0,1,0,0.516893
5051,167908400.0,225.0,78.720000,197.0,55.969543,153.0,35.241830,2.0,186.000000,NaN,...,11.0,11.0,7.0,2.0,1.0,0.0,0,1,0,0.421892
1917,154588624.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,0,0,1,0.325336
5793,168135504.0,1224.0,185.109477,876.0,190.592466,466.0,214.375536,220.0,232.154545,NaN,...,36.0,34.0,30.0,30.0,30.0,0.0,1,0,0,0.784467
4197,167652736.0,542.0,87.518450,163.0,186.061350,113.0,191.929204,49.0,190.918367,15.0,...,3.0,2.0,NaN,NaN,NaN,0.0,0,1,0,0.602872
3653,165855072.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,1,0,0,0.418187
7609,168691952.0,695.0,48.579856,176.0,136.784091,113.0,166.929204,75.0,151.120000,1.0,...,70.0,44.0,38.0,38.0,38.0,0.0,1,0,0,0.235198
528,137149152.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,1,0,0,0.604693


In [202]:
all_feature_dd.to_csv("D:/MusicFile/all_feature_dd_8823.csv", header = True, index = False)

In [203]:
all_feature_nd.to_csv("D:/MusicFile/all_feature_nd_8823.csv", header = True, index = False)

In [204]:
all_feature_total.to_csv("D:/MusicFile/all_feature_total_8823.csv", header = True, index = False)